In [1]:
import tensorflow as tf
import numpy as np
import os
import ast
import abc
import random
from typing import List, Dict, Union, Tuple
import numpy as np
from scipy.stats import truncnorm
import argparse
import timeit
from datetime import datetime
# from tensorflow_addons.layers.nalu import NALU
%load_ext tensorboard

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # disable cuda sepeed up
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # disable CPU wornings

2022-11-02 12:11:06.443774: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-02 12:11:06.443823: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("-o", "--output", dest="output", default="naly_syn_simple_arith")
parser.add_argument("-d", "--dist", dest="dist", default="normal", help="Prob.Dist")
parser.add_argument("-p", "--params",dest="params" , default="(-3,3)", type=ast.literal_eval)
parser.add_argument("-e", "--ext",dest="ext" , default="(10,15)", type=ast.literal_eval)

parser.add_argument("-n", "--nalu", dest="nalu", default="nalui1")
parser.add_argument("-se", "--seed", dest="seed", default=42, type=int)
parser.add_argument("-op", "--operation", dest="op", default="MUL")



args = parser.parse_args([])

def sample(dist, params, numDim = 2, numDP = 64000):
    data = np.zeros(shape=(numDP, numDim))
    if dist == "normal":
        intmean = (params[0] + params[1]) / 2
        intstd = (params[1] - params[0]) / 6
        print(
            "Generating Data: \nInt: \tdist \t %s\n\t\tdata >=\t %s\n\t\tmean(s)\t %s\n\t\tdata <\t %s\n\t\tstd \t %s" % (
                dist, params[0], intmean, params[1], intstd))
        mi, ma = (params[0] - intmean) / intstd, (params[1] - intmean) / intstd
        data = np.reshape(truncnorm.rvs(mi, ma, intmean, intstd, size=numDim * numDP), data.shape)

    elif dist == "uniform":
        print("Generating Data: \nInt: \tdist \t %s\n\t\tdata >=\t %s\n\t\tdata <\t %s\n\t\t" % (
        dist, params[0], params[1]))
        data = np.reshape(np.random.uniform(params[0], params[1], size=numDim * numDP), data.shape)
    elif dist == "exponential":
        data = np.random.exponential(params, size=(numDP, numDim))
    else:
        raise Exception("Unknown distribution")
    data = np.reshape(data, [-1])  # reshape to mix both distributions per instance!
    np.random.shuffle(data)
    data = np.reshape(data, (numDP, numDim))
    return data


def operation(op, a, b):
    if op.lower() == "mul":
        return a * b
    if op.lower() == "add":
        return a + b
    if op.lower() == "sub":
        return a - b
    if op.lower() == "div":
        return a / b

random.seed(args.seed)
tf.random.set_seed(args.seed)
np.random.seed(args.seed)

data = sample(args.dist, args.params)
lbls = operation(args.op, data[:,0], data[:,1])
lbls = np.reshape(lbls, newshape=(-1, 1))

int_data = sample(args.dist, args.params)
int_lbls = operation(args.op, int_data[:,0], int_data[:,1])
int_lbls = np.reshape(int_lbls, newshape=(-1, 1))

ext_data = sample(args.dist, args.ext)
ext_lbls = operation(args.op, ext_data[:,0], ext_data[:,1])
ext_lbls = np.reshape(ext_lbls, newshape=(-1, 1))

Generating Data: 
Int: 	dist 	 normal
		data >=	 -3
		mean(s)	 0.0
		data <	 3
		std 	 1.0
Generating Data: 
Int: 	dist 	 normal
		data >=	 -3
		mean(s)	 0.0
		data <	 3
		std 	 1.0
Generating Data: 
Int: 	dist 	 normal
		data >=	 10
		mean(s)	 12.5
		data <	 15
		std 	 0.8333333333333334


In [3]:
BATCH_SIZE = 64

data_dp = tf.data.Dataset.from_tensor_slices((data, lbls)).prefetch(tf.data.AUTOTUNE).batch(BATCH_SIZE)
int_data_dp = tf.data.Dataset.from_tensor_slices((int_data, int_lbls)).prefetch(tf.data.AUTOTUNE).batch(BATCH_SIZE)
ext_data_dp = tf.data.Dataset.from_tensor_slices((ext_data, ext_lbls)).prefetch(tf.data.AUTOTUNE).batch(BATCH_SIZE)

2022-11-02 12:11:13.616703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-02 12:11:13.616737: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-02 12:11:13.616764: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (filip-HP-ProBook-440-G3): /proc/driver/nvidia/version does not exist
2022-11-02 12:11:13.617659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Copyright 2020 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =====================================================epoch
import tensorflow as tf
from typeguard import typechecked
from typing import List
from tensorflow_addons.utils import types


class NALURegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, reg_coef=0.1):
        self.reg_coef = reg_coef

    def __call__(self, var: List[tf.Variable]) -> tf.Tensor:
        return self.reg_coef * tf.reduce_mean(
            tf.math.maximum(tf.math.minimum(-var, var) + 20, 0)
        )

    def get_config(self):
        return {"reg_coef": float(self.reg_coef)}


# @tf.keras.utils.register_keras_serializable(package="Addons")
class NALU(tf.keras.layers.Layer):
    r"""Neural Arithmetic Logic Units

    A layer that learns addition,substraction, multiplication and division 
    in transparent way. They layer has two paths: one for addition/substration 
    and one for multiplication/division. We can inspect weights for these two
    paths by calling `w_hat` and `m_hat` respectively. To use this layer reliably, 
    we have to delay regularization of gating varaible that switch between two paths.
    Ithave to be done by callback as from the layer-level we keep no information about epochs.


    See [Neural Arithmetic Logic Units](https://arxiv.org/abs/1808.00508)
    and [Improved Neural Arithmetic Logic Unit](https://arxiv.org/abs/2003.07629) 


    Example:

    >>> BATCH_SIZE, INPUT_SIZE, OUTPUT_SIZE = 16, 5, 2
    >>> input = tf.random.uniform((BATCH_SIZE, INPUT_SIZE))
    >>> nalu_layer = NALU(OUTPUT_SIZE)
    >>> predict = nalu_layer(input)
    >>> assert predict.shape == (BATCH_SIZE, OUTPUT_SIZE)

    Args:
        input_dim (int): input 
        output_dim (int): _description_
        regularizer (types.Regularizer, optional): _description_. Defaults to NALURegularizer(reg_coef=0.05).
        gate_as_vector (bool, optional): _description_. Defaults to True.
        clipping (float, optional): _description_. Defaults to None.
        force_operation (str, optional): _description_. Defaults to None.
        weights_separation (bool, optional): _description_. Defaults to True.
        input_gate_dependance (bool, optional): _description_. Defaults to True.
        w_initializer (types.Initializer, optional): _description_. Defaults to tf.random_normal_initializer( mean=0.88, stddev=0.2, seed=None ).
        m_initializer (types.Initializer, optional): _description_. Defaults to tf.random_normal_initializer( mean=0.5, stddev=0.2, seed=None ).
        g_initializer (types.Initializer, optional): _description_. Defaults to tf.random_normal_initializer( mean=0.0, stddev=0.2, seed=None ).
    """
    
    @typechecked
    def __init__(
        self,
        output_dim: int,
        regularizer: types.Regularizer = NALURegularizer(reg_coef=0.05),
        gate_as_vector: bool = True,
        clipping: float = None,
        force_operation: str = None,
        weights_separation: bool = True,
        input_gate_dependance: bool = True,
        w_initializer: types.Initializer = tf.random_normal_initializer(
            mean=0.88, stddev=0.2, seed=None
        ),
        m_initializer: types.Initializer = tf.random_normal_initializer(
            mean=0.5, stddev=0.2, seed=None
        ),
        g_initializer: types.Initializer = tf.random_normal_initializer(
            mean=0.0, stddev=0.2, seed=None
        ),
        *args,
        **kwargs,
    ):
        super(NALU, self).__init__(*args, **kwargs)

        self.reg_fn = regularizer
        self.output_dim = output_dim
        self.gate_as_vector = gate_as_vector
        self.clipping = clipping
        self.force_operation = force_operation
        self.weights_separation = weights_separation
        self.input_gate_dependance = input_gate_dependance
        self.w_initializer = w_initializer
        self.m_initializer = m_initializer
        self.g_initializer = g_initializer

    def build(self, input_shape):
        self.input_dim = int(input_shape[-1])
        if self.output_dim is None:
            raise ValueError('The last dimension of the inputs to `NALU` '
                            'should be defined. Found `None`.')

        # if-else statement is evaluated at the graph construction time, so no need to use tf.cond
        self.operator_weight_shape = (self.input_dim, self.output_dim) if self.gate_as_vector else (self.output_dim, self.input_dim)
        self.gate_shape = (self.input_dim, self.output_dim if self.gate_as_vector else 1) if self.input_gate_dependance else (self.output_dim,)

        # action variables
        self.w_hat = self.add_weight(
            name = "w",
            shape = self.operator_weight_shape, 
            dtype = tf.float32,
            initializer = self.w_initializer,
            regularizer = self.reg_fn, 
            trainable=True,
        )

        self.m_hat = self.add_weight(
            name = "m",
            shape = self.operator_weight_shape,
            dtype = tf.float32,
            initializer = self.m_initializer,
            regularizer = self.reg_fn, 
            trainable=True,
        )
        
        # extra weight for operation learning separation
        if self.weights_separation:
            self.w_hat_prime = self.add_weight(
                name = "w_prime",
                shape =self.operator_weight_shape,
                dtype = tf.float32,
                initializer = self.w_initializer,
                regularizer = self.reg_fn, 
                trainable=True,
            )

            self.m_hat_prime = self.add_weight(
                name = "m_prime",
                shape = self.operator_weight_shape,
                dtype = tf.float32,
                initializer = self.m_initializer,
                regularizer = self.reg_fn, 
                trainable=True,
            )

        # gating varaible

        self.g = self.add_weight(
            name = "g",
            shape = self.gate_shape, 
            dtype = tf.float32,
            initializer = self.g_initializer,
            regularizer = self.reg_fn, 
            trainable=False,
        )

    def call(self, input):
        eps = 1e-7
        w1 = tf.math.tanh(self.w_hat) * tf.math.sigmoid(self.m_hat)
        w1_prime = (
            tf.math.tanh(self.w_hat_prime) * tf.math.sigmoid(self.m_hat_prime)
            if self.weights_separation
            else tf.ones((1, 1))
        )  # dummy vector
        w1 = w1 if self.gate_as_vector else tf.transpose(w1)
        w1_prime = w1_prime if self.gate_as_vector else tf.transpose(w1_prime)
        a1 = tf.matmul(input, w1)
        m1 = tf.matmul(
            tf.math.log(tf.math.abs(input) + eps),
            w1_prime if self.weights_separation else w1,
        )
        m1 = tf.math.exp(tf.minimum(m1, self.clipping) if self.clipping else m1)
        g1 = tf.math.sigmoid(
            tf.matmul(input, self.g) if self.input_gate_dependance else self.g
        )
        if self.clipping:
            # sign correction
            ws = tf.math.abs(
                tf.reshape(w1_prime if self.weights_separation else w1, [-1])
            )
            xs = tf.concat([input] * w1.shape[1], axis=1)
            xs = tf.reshape(xs, shape=[-1, w1.shape[0] * w1.shape[1]])
            sgn = tf.sign(xs) * ws + (1 - ws)
            sgn = tf.reshape(sgn, shape=[-1, w1.shape[1], w1.shape[0]])
            ms = tf.math.reduce_prod(sgn, axis=2)
            m1 = m1 * tf.clip_by_value(ms, -0.1, 0.1)

        if not self.force_operation:
            return g1 * a1 + (1 - g1) * m1
        elif self.force_operation.lower() == "add":
            return a1
        elif self.force_operation.lower() == "mul":
            return m1
        else:
            # raise ValueError(f"force_operation must be: None/add/mul, but was: {self.force_operation}")
            raise tf.errors.InvalidArgumentError(
                f"force_operation must be: None/'add'/'mul', but was: {self.force_operation}"
            )

    def reinitialise(self):
        self.g.assign(tf.random.uniform(self.g.shape, -2, 2))
        self.w_hat.assign(tf.random.uniform(self.w_hat.shape, -2, 2))
        self.m_hat.assign(tf.random.uniform(self.m_hat.shape, -2, 2))
        if self.weights_separation:
            self.w_hat_prime.assign(tf.random.uniform(self.w_hat_prime.shape, -2, 2))
            self.m_hat_prime.assign(tf.random.uniform(self.m_hat_prime.shape, -2, 2))


    def get_gates_variables(self) -> List[tf.Variable]:
        return [self.g]


In [5]:

# check if NALU layer returns expected shape in all possible configurations
input = next(iter(data_dp))[0]
i = 1
for gate_as_vector in [True, False]:
    for clipping in [None, 20]:
        for force_operation in [None, "add", "mul"]:
            for weights_separation in [False, True]:
                for input_gate_dependance in [False, True]:
                    layer = NALU(
                        i,
                        gate_as_vector = gate_as_vector,
                        clipping = clipping,
                        force_operation = force_operation,
                        weights_separation = weights_separation,
                        input_gate_dependance = input_gate_dependance
                    )
                    assert layer(input).shape == (BATCH_SIZE, i)
                    i+=1

In [57]:
class NALUModel(tf.keras.Model):

    def __init__(self, *args, **kwargs):
        super(NALUModel, self).__init__(*args, **kwargs)
        self.layer = NALU(1)        

        self.steps_counter = tf.Variable(0, trainable=False)
        self.epoch_counter = tf.Variable(0, trainable=False)
        self.reinitialization_counter = tf.Variable(0, trainable=False)
        self.regularize = tf.Variable(False, trainable=False)
        self.gating = tf.Variable(False, trainable=False)
        self.gate_var = None

    def reinitialise(self):
        for l in self.layers:
            if isinstance(l, NALU):
                l.reinitialise()


    def get_gates_variables(self) -> List[tf.Variable]:
        return [l.g for l in self.layers if isinstance(l, NALU)]

    def get_regularization_loss(self):
        return tf.math.reduce_sum(self.losses)

    @tf.function
    def train_step_active(self, data):
        x, y = data
        with tf.GradientTape(watch_accessed_variables = True) as tape:
            logits = self(x, training=True) 
            loss_value = self.compiled_loss(y, logits)
            loss_value = tf.math.add(loss_value, tf.cond(
                tf.math.logical_and(self.regularize, tf.math.less(loss_value, 1.0)),
                lambda: self.get_regularization_loss(), 
                lambda: tf.constant(0, dtype = tf.float32)))
        
        grads = tape.gradient(loss_value, tape.watched_variables())
        self.optimizer.apply_gradients(zip(grads, tape.watched_variables()))
        self.compiled_metrics.update_state(y, logits)
        return  {m.name: m.result() for m in self.metrics}

    @tf.function
    def train_step_gating(self, data):
        if not self.gate_var:
            self.gate_var = self.get_gates_variables()

        x, y = data
        with tf.GradientTape(watch_accessed_variables = False) as tape:
            for v in self.gate_var:
                tape.watch(v)
            logits = self(x, training=True) 
            loss_value = self.compiled_loss(y, logits)
            loss_value = tf.math.add(loss_value, tf.cond(
                tf.math.logical_and(self.regularize, tf.math.less(loss_value, 1.0)),
                lambda: self.get_regularization_loss(), 
                lambda: tf.constant(0, dtype = tf.float32)))

        grads = tape.gradient(loss_value, tape.watched_variables())
        self.optimizer.apply_gradients(zip(grads, tape.watched_variables()))
        self.compiled_metrics.update_state(y, logits)
        return  {m.name: m.result() for m in self.metrics}

    def call(self, inputs: tf.Tensor, training: bool = True):
        return  self.layer(inputs) # self.layer3(self.layer2(self.layer(inputs)))

    @tf.function
    def train_step(self, data):
        """
        Specifying tf.function(input_signature=...) slows down the computation, but it leads to greater control:
        https://www.neuralconcept.com/post/in-graph-training-loop
        
        """
        print("tracking check")
        
        return tf.cond(self.gating, 
            lambda: self.train_step_active(data),
            lambda: self.train_step_gating(data))

In [135]:
seed = args.seed
g_intializer = tf.random_normal_initializer(mean=0, stddev=0.1, seed=seed)
w_intializer = tf.random_normal_initializer(mean=1, stddev=0.1, seed=seed)
m_intializer = tf.random_normal_initializer(mean=-1, stddev=0.1, seed=seed)
batch_size = 64
clipping = 20
lr = 0.01
reg_coef = 0.05

tmp = None


class Superc(tf.keras.callbacks.Callback):
    
    reinit_history = []
    reinit_counter = 0
    _steps_counter = 0
    _epoch_counter = 0
    epochs_to_reg = 5
    lt_to_reg = 1.0
    fr_param_check = 5

    # inspect why __init__ does not work for callback subclass

    # def __init_(self, epochs_to_reg: int = 5, lt_to_reg: float = 1.0, fr_param_check: int = 5, *args, **kwargs):
    #     super(Superc, self).__init__(*args, *kwargs)
    #     Superc.epochs_to_reg = 5
    #     Superc.lt_to_reg = lt_to_reg
    #     Superc.fr_param_check = fr_param_check

    
    # delay regularize
    def on_train_epoch_end(self, epoch, logs = None):
        Superc._epoch_counter += 1

    def on_train_batch_end(self, batch, logs = None):
        
        Superc._steps_counter += 1
        
        # record last loss
        Superc.reinit_history.append(logs.get("loss"))

        # train either active or gating
        self.model.gating.assign(Superc._steps_counter % 10 < 8)

        # turn on or of ragularization depending on epoch number and last seen loss
        self.model.regularize.assign(Superc._epoch_counter > Superc.epochs_to_reg and Superc.reinit_history[-1] < Superc.lt_to_reg)


        # reinitialisation strategy
        split_index = len(Superc.reinit_history)//2
        if len(Superc.reinit_history)>10000 \
            and Superc._epoch_counter > 0 \
            and Superc._epoch_counter%10==1 \
            and tf.math.reduce_mean(Superc.reinit_history[:split_index]) <= tf.math.add(
                        tf.math.reduce_mean(Superc.reinit_history[split_index:]),
                        tf.math.reduce_std(Superc.reinit_history[split_index:])
                        ) \
            and tf.math.reduce_mean(Superc.reinit_history[split_index:]) > 1:
            # reinitialize all nalu layers 
            self.model.reinitialise()
            Superc.reinit_history = []
            Superc.reinit_counter += 1

        # check parameters
        if self._steps_counter % 1500 ==0:
            tf.print("\n--- Parameter Check Start---")
            eloss_ex = self.model.compiled_loss._losses[0](self.model.predict(ext_data_dp, verbose = 1), ext_lbls)
            eloss_in = self.model.compiled_loss._losses[0](self.model.predict(int_data_dp, verbose = 1), int_lbls)
            tf.print("--- Parameter Check End---")

        


In [136]:
     
mm = NALUModel()
mm.compile(optimizer="adam", loss="mse", metrics=["mae"])
# print(mm.variables)

metrtics = mm.fit(
    data_dp,
    epochs = 3, 
    batch_size = batch_size, 
    callbacks=[
        Superc()
        ])


Epoch 1/3
tracking check
1000/1000 [==============================] - 6s 3ms/step - loss: 1.1165 - mae: 0.6848
Epoch 2/3
 496/1000 [=============>................] - ETA: 1s - loss: 1.0637 - mae: 0.6519
--- Parameter Check Start---
1000/1000 [==============================] - 2s 2ms/step
--- Parameter Check End---
1000/1000 [==============================] - 7s 7ms/step - loss: 1.0751 - mae: 0.6538
Epoch 3/3
 995/1000 [============================>.] - ETA: 0s - loss: 1.0621 - mae: 0.6462
--- Parameter Check Start---
1000/1000 [==============================] - 2s 2ms/step
--- Parameter Check End---
1000/1000 [==============================] - 7s 7ms/step - loss: 1.0616 - mae: 0.6460


In [ ]:
# garbage collector

In [37]:
# with open("%s.csv" % "_".join([args.output, args.nalu, args.dist, str(args.params), str(args.ext), str(args.seed), args.op, "int"]), "w") as intlog:
#     with open("%s.csv" % "_".join([args.output, args.nalu, args.dist, str(args.params), str(args.ext), str(args.seed), args.op, "ext"]), "w") as extlog:
losses = []
reinitctr = 0
for epoch in range(10):
    print("Epoch:",epoch)
    for i, data in enumerate(data_dp):
        if i % 10 < 8:
            loss_value = mm.train_step_active(data)["loss"]
        else:
            loss_value = mm.train_step_gating(data)["loss"]

        if REGULARIZATION:
            if epoch > 10 and loss_value < 1:
                mm.regularize.assign(True)
                loss_value = mm.train_step_active(data)["loss"]
                loss_value = mm.train_step_gating(data)["loss"]
            else:
                mm.regularize.assign(False)
        losses.append(loss_value)

        # reinitialisation strategy
        split_index = len(losses)//2
        if split_index>10000//BATCH_SIZE \
            and epoch > 0 \
            and epoch%10==1 \
            and tf.math.reduce_mean(losses[:split_index]) <= tf.math.add(
                        tf.math.reduce_mean(losses[split_index:]),
                        tf.math.reduce_std(losses[split_index:])
                        ) \
            and tf.math.reduce_mean(losses[split_index:]) > 1:
            mm.reinitialise()
            losses = []
            reinitctr += 1

        if epoch > 5:
            if epoch > 7:
                good = 0
                for x, y in zip(mm.predict(data_dp), data_dp[-1]):
                    if abs(x - y)< 1e-5:
                        good += 1
                print(f"good: {good}")
            eloss_ex = mm.compiled_loss(mm.predict, ext_data_dp[-1]) # np.mean([mm.test_step(x)["loss"] for x in ext_data_dp])
            print(eloss_ex)
            # eloss_in = np.mean([mm.test_step(x)["loss"] for x in int_data_dp])

            # print("int loss: {:.5E}\tregularization-loss: {:.5E}".format(eloss_in, erloss_in))
            
            # intlog.write("\t".join([str(epoch), str(i), args.output, args.nalu, args.dist, str(args.params), str(args.ext), str(args.seed), args.op, "{:.5E}".format(eloss), "{:.5E}".format(eloss_in), str(reinitctr)])+"\n")
            # extlog.write("\t".join([str(epoch), str(i), args.output, args.nalu, args.dist, str(args.params), str(args.ext), str(args.seed), args.op, "{:.5E}".format(eloss), "{:.5E}".format(eloss_ex), str(reinitctr)]) + "\n")


<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [38]:

#  PREVIOUS CALLBACK KEEP FOR INSPECTION

class DelayRegularize(tf.keras.callbacks.Callback):
    regularize_epochs = 5    
    def on_epoch_end(self, epoch, logs=None):
        tf.cond(tf.math.greater(epoch, DelayRegularize.regularize_epochs), lambda: self.model.regularize.assign(True), lambda: None)
        

class ReinitializatoinControll(tf.keras.callbacks.Callback):

    history = []
    counter = 0
    
    def on_train_batch_end(self, batch, logs=None):
        ReinitializatoinControll.history.append(logs.get("loss"))
    

    def on_epoch_end(self, epoch, logs=None):
        split_index = len(ReinitializatoinControll.history)//2
        if split_index>500 \
            and epoch > 0 \
            and epoch%10==1 \
            and tf.math.reduce_mean(self.history[:split_index]) <= tf.math.add(
                        tf.math.reduce_mean(self.history[split_index:]),
                        tf.math.reduce_std(self.history[split_index:])
                        ) \
            and tf.math.reduce_mean(ReinitializatoinControll.history[split_index:]) > 1:
            self.model.reinitialise()
            ReinitializatoinControll.history = []
            ReinitializatoinControll.counter += 1


class Evaluator(tf.keras.callbacks.Callback):
    history = []
    def on_test_batch_end(self, epoch, logs = None):
        if tf.math.mod(self.model.steps_counter, 10)==0:
            Evaluator.history.append(logs.get("loss"))

class EXTCSV(tf.keras.callbacks.Callback):
    counter = 0

    def on_train_begin(self, logs = None):
        pass
    
    def on_train_epoch_end(self, epoch, logs = None):
        EXTCSV.counter = epoch

    def on_train_batch_end(self, batch, logs = None):
        if EXTCSV.counter*64 + batch

<tf.Tensor: shape=(), dtype=bool, numpy=True>